In [1]:
import csv # csv file format 달기 위함
import random
import numpy as np
import tensorflow as tf

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Preprocess data : names

In [2]:
data_name = set()      # name set
max_len = 0            # maximum length
with open('../data/woman_name_dataset.csv') as csv_file: 
    csv_reader = csv.reader(csv_file, delimiter=',') 
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1      # 첫 line은 컬러명이라서 의미가 없어서.
        else:
            tmp_name = row[1].split()[0]        # split은 중간의 공백을 구별해서. 리스트로 반납   rddrrrr r
            data_name.add(row[1].split()[0])
            if len(tmp_name) > max_len:
                max_len = len(tmp_name)         # 데이터 중 최대의 길이로 셋팅
data_name = list(data_name)                     # set보다는 list가 편해서..
print('name total : {}'.format(len(data_name)))
print('maximum name length : {}'.format(max_len))    

name total : 1219
maximum name length : 11


## Preprocess data : Characters

In [3]:
# 문자를 숫자로.. one hot vector로   preprocessing
chars = set()     # = {a,b,.....,z}
for name in data_name:
    for char in name:
        chars.add(char)
chars = list(np.sort(list(chars)))      # no.sort는   numpy array 로 되어서
print('{} alphabets : '.format(len(chars)), chars)

26 alphabets :  ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## Define function to convert name to onehot

In [4]:
def name_to_onehot(names, chars, max_len):
    # len(names) : batch size
    onehot = np.zeros((len(names), max_len, len(chars)+1))      #  이름이 끝나는 것에  signal을 주고 싶어서 Tom마지막 , placeholder에 같은 크기
    for idx_1, name in enumerate(names):           # idx_1는 이름의 갯수
        for idx_2 in range(max_len):               # idx_2는 이름의 char 갯수
            if idx_2 < len(name):
                idx_3 = chars.index(name[idx_2])  # idx3 전체 albartbet의 몇번째인가.
                onehot[idx_1, idx_2, idx_3] = 1
            else:
                onehot[idx_1, idx_2, -1] = 1       #  -1 을 주면  onehot이 끝나는것
    return onehot

onehot_ex = name_to_onehot(['jane'], chars, max_len)

## Define dimension and Placeholders

In [5]:
num_data = len(data_name)
seq_len = max_len - 1
dim_data = len(chars) + 1   #  size of the one-hot vecotr.

ph_input_name = tf.placeholder(dtype=tf.float32, shape=[None, seq_len, dim_data])   # batch size, maximum seqence 길이,  onehot 길이
ph_output_name = tf.placeholder(dtype=tf.float32, shape=[None, seq_len, dim_data])

## Define weight variables

In [6]:
dim_rnn_cell = 128   # hideen layer를  ......
stddev = 0.02

with tf.variable_scope('weights'):   # variable_scope를 담는 통 ( 일반적인 w1, w2, w3를 쓰면..헷갈리단..  variable_scope안의 w1, w2, w3)
    W_i = tf.get_variable('W_i', dtype=tf.float32,
                         initializer=tf.random_normal([dim_data, dim_rnn_cell],
                         stddev = stddev))
    # data : 1 by dim_data
    # W_i : dim_data by dim_rnn_cell
    b_i = tf.get_variable('b_i', dtype=tf.float32,
                         initializer=tf.random_normal([dim_rnn_cell],
                         stddev = stddev))
    
    # b_i : 1 by dim_rnn_cell
    # h= data * W_i * b_i : batch by dim_data * dim_data by dim_rnn cell +1 1 by dim_rnn_cell
    # h : 1 by dim_rnn_cell
    W_o = tf.get_variable('W_o', dtype=tf.float32,
                         initializer=tf.random_normal([dim_rnn_cell, dim_data],
                         stddev = stddev))
    b_o = tf.get_variable('b_o', dtype=tf.float32,
                         initializer=tf.random_normal([dim_data],
                         stddev = stddev))
    
    # LSTM : batch by dim_rnn_cell : batch  by dim_data
    # I want one hot encoding vector!
    # LSTM CELL * W_o(dim_rnn_cell by dim_data) + b_o(dim_data)

## Define RNN for training

In [11]:
#n_dim_rnn_cell  : LSTM Hidden Cell Size
def name_rnn_train(_x, _seq_len, _dim_data, _dim_rnn_cell):
    # _x : ph_input_name : Batch(0), seq_len(1), dim_data(2)
    _x_split = tf.transpose(_x, [1, 0, 2]) # seq_len, batch, dim_data
    _x_split = tf.reshape(_x_split, [-1, _dim_data])
    
    # x_split : seq_len*batch by dim_data
    # use tf.AUTO_REUSE,
    # Load Variables
    with tf.variable_scope('weights', reuse= tf.AUTO_REUSE):
        _W_i = tf.get_variable('W_i')
        _b_i = tf.get_variable('b_i')
        _W_o = tf.get_variable('W_o')
        _b_o = tf.get_variable('b_o')
    # Linear Operation for Input
    _h_split = tf.matmul(_x_split, _W_i) + b_i
    _h_split = tf.split(_h_split, _seq_len, axis=0)
    
    # Define LSTM Cell && RNN
    with tf.variable_scope('rnn', reuse=tf.AUTO_REUSE):
        _rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(_dim_rnn_cell)
        _output, _state = tf.nn.static_rnn(_rnn_cell, _h_split, dtype=tf.float32)
    
    _total_out = []
    
    for _tmp_out in _output:
        _tmp_out = tf.matmul(_tmp_out, _W_o) + _b_o
        _total_out.append(_tmp_out)
        
    return tf.transpose(tf.stack(_total_out), [1, 0, 2])

## Define result graph

In [12]:
result_name = name_rnn_train(ph_input_name, seq_len, dim_data, dim_rnn_cell)
print('result_shape :', result_name.shape)

result_shape : (?, 10, 27)


## Define Loss function

In [14]:
def name_loss(_gt_name, _result_name, _seq_len):
    total_loss = 0
    # _resutl_name  : batch, seq_len, dim_data
    # ->bathc, dim_data
    # batch by dim_data -> loss calculate
    for i in range(_seq_len):
        gt_char = _gt_name[:, i, :]          # batch, dim_data
        result_char = _result_name[:, i, :] # batch, dim_data
        tmp_loss = tf.nn.softmax_cross_entropy_with_logits(labels=gt_char, 
                                                           logits=result_char)
        tmp_loss = tf.reduce_mean(tmp_loss)
        total_loss += tmp_loss
    return total_loss
rnn_loss = name_loss(ph_output_name, result_name, seq_len)

## Define Optimizer and Get Ready

In [15]:
learning_rate = 1e-3
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(rnn_loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list=tf.trainable_variables())   # 학습이 될수 있는 variabale 이 무엇이나..

print('Now ready to start the session')

Now ready to start the session


## Session Run

In [16]:
max_epoch = 300
batch_size = 64
num_batch = int(num_data/batch_size)
with tf.Session() as sess:
    sess.run(init)
    
    for _epoch in range(max_epoch):
        random.seed(_epoch)
        batch_shuffle = list(range(num_data))
        random.shuffle(batch_shuffle)
        
        total_train_loss = 0
        
        for i in range(num_batch):
            batch_idx = [batch_shuffle[idx] for idx in range(i*batch_size,
                                                            (i+1)*batch_size)]
            batch_names = [name for name in data_name if data_name.index(name) in batch_idx]
            batch_onehots = name_to_onehot(batch_names, chars, max_len)
            
            input_onehot = batch_onehots[:, 0:(max_len-1), :]   # a b y s
            output_onehot = batch_onehots[:, 1:max_len, :]      # b y s

            train_feed_dict = {ph_input_name: input_onehot, 
                               ph_output_name: output_onehot}
            
            sess.run(optimizer, feed_dict = train_feed_dict)
            curr_loss = sess.run(rnn_loss, feed_dict=train_feed_dict)
            
            total_train_loss += curr_loss/num_batch

        print('epoch : {}, train_loss : {}'.format(_epoch+1, total_train_loss))
        
        model_save_path = saver.save(sess, './RNN_model/model.ckpt', global_step=_epoch+1)
        print('Model saved in file: {}'.format(model_save_path))

epoch : 1, train_loss : 28.88234760886744
Model saved in file: ./RNN_model/model.ckpt-1
epoch : 2, train_loss : 18.6269992025275
Model saved in file: ./RNN_model/model.ckpt-2
epoch : 3, train_loss : 17.659534353958936
Model saved in file: ./RNN_model/model.ckpt-3
epoch : 4, train_loss : 17.120001742714333
Model saved in file: ./RNN_model/model.ckpt-4
epoch : 5, train_loss : 16.72336307324861
Model saved in file: ./RNN_model/model.ckpt-5
epoch : 6, train_loss : 16.349830225894326
Model saved in file: ./RNN_model/model.ckpt-6
epoch : 7, train_loss : 16.048421508387516
Model saved in file: ./RNN_model/model.ckpt-7
epoch : 8, train_loss : 15.76255958958676
Model saved in file: ./RNN_model/model.ckpt-8
epoch : 9, train_loss : 15.47641242177863
Model saved in file: ./RNN_model/model.ckpt-9
epoch : 10, train_loss : 15.2146880501195
Model saved in file: ./RNN_model/model.ckpt-10
epoch : 11, train_loss : 14.931576126500183
Model saved in file: ./RNN_model/model.ckpt-11
epoch : 12, train_loss : 

## Define RNN to test

In [17]:
ph_test_input_name = tf.placeholder(dtype=tf.float32, shape=[1, 1, dim_data])
# hideen  size : 1 
ph_h = tf.placeholder(dtype=tf.float32, shape=[1, dim_rnn_cell])
# hideen stat of LSTM
ph_c = tf.placeholder(dtype=tf.float32, shape=[1, dim_rnn_cell])
# cell state of LSTM

def name_rnn_test(_x, _dim_data, _dim_rnn_cell, _prev_h, _prev_c):  # ph_h, ph_c
    _x_split = tf.transpose(_x, [1, 0, 2]) # seq_len, batch, dim_data
    _x_split = tf.reshape(_x_split, [-1, _dim_data])
    
    with tf.variable_scope('weights', reuse=tf.AUTO_REUSE):
        _W_i = tf.get_variable('W_i')
        _b_i = tf.get_variable('b_i')
        
        _W_o = tf.get_variable('W_o')
        _b_o = tf.get_variable('b_o')
        
    _h_split = tf.matmul(_x_split, _W_i) + b_i
    _h_split = tf.split(_h_split, 1, axis=0) # 1 is the seq_len
    
    with tf.variable_scope('rnn', reuse=tf.AUTO_REUSE):
        _rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(_dim_rnn_cell)
        _output, _state = tf.nn.static_rnn(_rnn_cell, _h_split, dtype=tf.float32,
                                           initial_state = (_prev_h, _prev_c))
    _total_out = []
    
    for _tmp_out in _output:
        _tmp_out = tf.matmul(_tmp_out, _W_o) + _b_o
        _total_out.append(_tmp_out)
        
    return tf.transpose(tf.stack(_total_out), [1, 0, 2]), _state  # output _state를다시 쓰기위하여

## Run Test

In [ ]:
test_result_name, test_state = name_rnn_test(ph_test_input_name, dim_data, dim_rnn_cell,
                                            ph_h, ph_c)

with tf.Session() as sess:
    sess.run(init)
    
    saver.restore(sess, './RNN_model/model.ckpt-300')
    
    total_name = ''
    prev_char = 'a'
    total_name += prev_char
    prev_state = (np.zeros((1, dim_rnn_cell)), np.zeros((1, dim_rnn_cell)))
    for i in range(seq_len):
        input_onehot = np.zeros((1, 1, dim_data)) # make a space
        prev_char_idx = chars.index(prev_char)
        input_onehot[:, :, prev_char_idx] = 1
            
        test_feed_dict = {ph_test_input_name: input_onehot,
                          ph_h: prev_state[0], ph_c: prev_state[1]}
        curr_result, curr_state = sess.run([test_result_name, test_state], test_feed_dict)
        if np.argmax(curr_result) == dim_data-1:
            break
        else:
            softmax_result = sess.run(tf.nn.softmax(test_result_name), test_feed_dict)
            softmax_result = np.squeeze(softmax_result)
            softmax_result = softmax_result[:dim_data-1]/sum(softmax_result[:dim_data-1])
            prev_char = np.random.choice(chars, 1, p=softmax_result)
            total_name += prev_char[0]
            prev_state = curr_state        
    print('Result Name :', total_name)